In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
uber_trips_2014 =pd.read_csv(r'D:\Documents\ML\Uber\uber_trips_2014.csv')
uber_trips_2015 =pd.read_csv(r'D:\Documents\ML\Uber\uber_trips_2015.csv')
weather = pd.read_csv(r'D:\Documents\ML\Uber\weather.csv')
geographic = pd.read_csv(r'D:\Documents\ML\Uber\geographic.csv')
green_trips_new_2 = pd.read_csv(r'D:\Documents\ML\Uber\green_trips_new_2.csv')
mta_trips = pd.read_csv(r'D:\Documents\ML\Uber\mta_trips.csv')
demographics = pd.read_csv(r'D:\Documents\ML\Uber\demographics.csv')
yellow_trips_new = pd.read_csv(r'D:\Documents\ML\Uber\yellow_trips_new.csv')
zones =  pd.read_csv(r'D:\Documents\ML\Uber\zones.csv')

In [97]:
mta_trips.sample(10)

,station,line_name,division,audit_type,unit_id,datetime,new_entries,new_exits,latitude,longitude
2233749,23 ST-6 AVE,FM,IND,REGULAR,R203_N509_00-00-00,04/29/2014 20:00:00,1438,727,40.742878,-73.992821
7178730,CENTRAL AVE,M,BMT,REGULAR,R401_K017_00-03-01,05/20/2015 21:00:00,216,158,40.697857,-73.927397
3942382,34 ST-PENN STA,ACE123,IND,REGULAR,R013_N073_02-00-05,06/15/2015 20:00:00,153,909,40.752287,-73.993391
3989980,5 AVE-53 ST,EM,IND,REGULAR,R015_N303_00-00-05,03/18/2015 18:00:00,620,44,40.645069,-74.014034
5954435,103 ST,1,IRT,REGULAR,R191_R170_00-03-01,12/31/2014 08:00:00,89,38,40.796092,-73.961454
2244486,CHURCH AVE,FG,IND,REGULAR,R204_N546_00-00-01,08/24/2014 16:00:00,117,119,40.650527,-73.962982
5723292,14 ST-UNION SQ,LNQR456,BMT,REGULAR,R170_A034_03-00-00,01/02/2015 08:00:00,18,146,40.734673,-73.989951
6143023,BAY RIDGE-95 ST,R,BMT,REGULAR,R216_C027_00-03-00,05/02/2015 12:00:00,75,56,40.634967,-74.023377
3372185,AVE U,N,BMT,REGULAR,R396_D015_00-00-02,08/03/2014 05:00:00,17,8,40.599300,-73.955929
2079115,BEDFORD PARK BL,4,IRT,REGULAR,R183_R291_00-00-02,03/29/2014 16:00:00,228,94,40.873244,-73.887138


In [85]:
list = ['QN25','BK43','BK37','BX63']
geo = pd.DataFrame(data=geographic,columns=list)
geo.to_csv(r'D:\Documents\ML\Uber\geo.csv')

In [87]:
geo_QN25 = geo['QN25'].dropna(axis=0)
geo_BK43 = geo['BK43'].dropna(axis=0)
geo_BK37 = geo['BK37'].dropna(axis=0)
geo_BX = geo['BX63'].dropna(axis=0)

In [93]:
x=0
for x in len(geo_QN25):
    if (x%2==0):
        list.append(x)

TypeError: 'int' object is not iterable

In [24]:
df['Year'] = [x[4:6] for x in df['pickup_datetime']]

In [80]:
#geo_QN25['LONGITUDE'] = [x for x in geo_QN25.values if x<0]
geo_QN25['LATITUDE'] = [x for x in geo_QN25.values if x>0]

TypeError: '>' not supported between instances of 'list' and 'int'

In [48]:
df_2 = df[df['Year']!="14"]
df_2.sample(10)

,pickup_datetime,pickup_latitude,pickup_longitude,base,Year
2654645,7/30/2014 8:19:00,40.7676,-73.9916,B02682,/2
2945499,8/2/2014 10:16:00,40.6934,-73.9995,B02617,20
696869,5/12/2014 14:23:00,40.7667,-73.9883,B02598,/2
1571824,6/14/2014 22:19:00,40.7485,-73.9731,B02617,/2
256165,4/10/14 21:32,40.7516,-73.9921,B02617,/1
3244846,8/28/2014 8:27:00,40.7651,-73.9782,B02617,/2
664873,5/8/2014 18:43:00,40.7284,-73.9879,B02598,20
3858995,9/6/2014 22:12:00,40.7619,-73.8295,B02617,20
3200533,8/23/2014 20:49:00,40.7056,-74.0086,B02617,/2
1913428,7/30/2014 11:59:00,40.7411,-73.9984,B02512,/2


In [99]:
df_copy_monthdays = df_copy.pivot_table(index=['Month'],
                                  values='BOROUGH',
                                  aggfunc='count')
df_copy_monthdays.plot(kind='bar', figsize=(8,6),colormap = 'rainbow')
plt.ylabel('Average')
plt.title('Injured by Month Day');

NameError: name 'df_copy' is not defined

In [36]:
df_1.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base,Year
0,4/1/14 0:11,40.7690,-73.9549,B02512,14
1,4/1/14 0:17,40.7267,-74.0345,B02512,14
2,4/1/14 0:21,40.7316,-73.9873,B02512,14
3,4/1/14 0:28,40.7588,-73.9776,B02512,14
4,4/1/14 0:33,40.7594,-73.9722,B02512,14


In [98]:
df_2['Date/Time'] = pd.to_datetime(df_2['pickup_datetime'], format="%m/%d/%Y %H:%M:%S")
df_2['DayOfWeekNum'] = df_2['Date/Time'].dt.dayofweek
df_2['DayOfWeek'] = df_2['Date/Time'].dt.weekday_name
df_2['MonthDayNum'] = df_2['Date/Time'].dt.day
df_2['HourOfDay'] = df_2['Date/Time'].dt.hour
df_2['Year'] = df_2['Date/Time'].dt.year

ValueError: time data '4/10/14 0:03' does not match format '%m/%d/%Y %H:%M:%S' (match)